In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("../input/hotel-booking-demand/hotel_bookings.csv")
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

since company having maximum missing value we are droping that

In [ ]:
df.drop('company',axis=1,inplace=True)

In [ ]:
df['hotel'].value_counts(normalize=True).plot(kind='bar')
plt.show()

here we can see that nearly 70% of hotel type is  city when compare to resort hotel

In [ ]:
df.groupby('hotel')['is_canceled'].value_counts(normalize=True).plot(kind='bar')
plt.show()

by observing the above graph we can see that maximum cancellation from hotel end is happened in city hotel than resort hotel

In [ ]:
df.groupby('is_canceled')['lead_time'].value_counts()

by seeing above we can see that approx if lead_time goes too much like more than 300 it leads to cancelation done by hotel end.

In [ ]:
plt.figure(figsize=(8,5))
df.groupby(['hotel'])['arrival_date_year'].value_counts(normalize=True).plot(kind='bar')
plt.show()

where we can see that maximum number of peoples arravied to hotel in both city and resort side are in year of 2016

In [ ]:
plt.figure(figsize=(8,5))
df.groupby(['hotel'])['stays_in_weekend_nights'].value_counts(normalize=True).plot(kind='bar')
plt.show()

we can see that mostly there is no stay on weekend night in city hotel where as in resort hotel they are more or less equal percentage of no stay are 2day stay 

In [ ]:
df.groupby(['arrival_date_month'])['stays_in_weekend_nights','stays_in_week_nights'].sum().plot(kind='bar')
plt.show()

In week day nights maximum booking done in August month and in week_end nights maximum booking done in Auguest and July

In [ ]:
plt.figure(figsize=(8,5))
df.groupby(['hotel'])['deposit_type'].value_counts(normalize=True).plot(kind='bar')
plt.show()

ther are many no refund deposit is in city side and there are many resort hotels are there which requried no deposit

In [ ]:
plt.figure(figsize=(8,5))
df[df['stays_in_week_nights']<=10].groupby(['hotel'])['stays_in_week_nights'].value_counts(normalize=True).plot(kind='bar')
plt.show()

1. in week days if there is plan of staying less than 2 or 3 days means most of them planning to stay in city hotel than resort hotel

2. vise versa if there is plan more than 4 days means many prefer to resort hotel than city side hotel

In [ ]:
plt.figure(figsize=(8,5))
df[df['adults']<=5].groupby(['hotel','adults'])['children'].value_counts(normalize=True).plot(kind='bar')
plt.show()

there is no major details with family since they are preffering both city and resort hotels

In [ ]:
df.groupby('hotel')['is_repeated_guest'].value_counts(normalize=True).plot(kind='bar')
plt.show()

in both case hotel they are very less possibality for repeated guest, might be all the peope in every country are planning to visit different hotel at each time

In [ ]:
df['reservation_status'].value_counts()

In [ ]:
df['reservation_status_date'].value_counts()

In [ ]:
df['customer_type'].value_counts()

In [ ]:
df['days_in_waiting_list'].value_counts()

In [ ]:
df[df['days_in_waiting_list']>5].groupby('hotel')['days_in_waiting_list'].value_counts()

In [ ]:
df[df['reserved_room_type'] != df['assigned_room_type']]['assigned_room_type'].count()

only bolow 10% people not get the reserved room type all others got the tyoe which they reserved itself

In [ ]:
df[df['previous_cancellations'] != df['previous_bookings_not_canceled']]['previous_bookings_not_canceled'].count()

nearly 8% of people where cancelled previous booking

In [ ]:
print(df[df['previous_cancellations']==1]['customer_type'].value_counts())
df[df['previous_cancellations']==1].groupby('hotel')['customer_type'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
df.info()

maximum transient type cumstomer are cancelled the perivious booking then the party that to in city hotel only they are maximum cancellasion happend

In [ ]:
from sklearn.preprocessing import LabelEncoder
lr = LabelEncoder()

In [ ]:
df['hotel'] = lr.fit_transform(df['hotel'])

In [ ]:
df = pd.get_dummies(df,columns=['reservation_status'],drop_first=True)

In [ ]:
df = pd.get_dummies(df,columns=['deposit_type','meal','market_segment','distribution_channel'],drop_first=True)

In [ ]:
df = pd.get_dummies(df,columns=['reserved_room_type','assigned_room_type','customer_type'],drop_first=True)

In [ ]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [ ]:
df['agent'].mode()

In [ ]:
df['children'].mode()

In [ ]:
df['agent'] = df['agent'].fillna(9.0)
df['children'] = df['children'].fillna(0.0)

In [ ]:
df1 = df.drop(['reservation_status_date'],axis=1)

In [ ]:
df1.info()

### Doing unseupervised learning only for country portugal(code : PRT) which having nearly 45% of overall data

In [ ]:
df2 = df1[df1['country']=='PRT']

In [ ]:
df2 = df2.drop('country',axis=1)

In [ ]:
df2 = pd.get_dummies(df2,columns=['arrival_date_month'],drop_first=True)

### Classification model regadring with you will get booking or not in PRT

In [ ]:
X = df2.drop(['is_canceled'],axis=1)
y = df2['is_canceled']

In [ ]:
X.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc= RandomForestClassifier()
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint


In [ ]:
params={'n_estimators':sp_randint(5,150), 'max_features':sp_randint(10,40),'max_depth':sp_randint(2,10),'min_samples_leaf':sp_randint(1,50),'min_samples_split':sp_randint(2,50),'criterion':['gini','entropy']}
rsearch=RandomizedSearchCV(rfc,param_distributions=params,n_jobs=-1,scoring='accuracy',n_iter=100,cv=3)
rsearch.fit(X,y)

In [ ]:
rsearch.best_params_

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)
rfc= RandomForestClassifier(**rsearch.best_params_)
rfc.fit(X_train,y_train)
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
y_train_pred=rfc.predict(X_train)
#y_train_prob=rfc.predict_proba(X_train)[:,1]
print('Accuracy score of train: ', accuracy_score(y_train,y_train_pred))
print('Confusion Matrix of train: ', confusion_matrix(y_train,y_train_pred))
#print('Auc of train: ', roc_auc_score(y_train,y_train_prob))
y_test_pred=rfc.predict(X_test)
#y_test_prob=rfc.predict_proba(X_test)[:,1]
print('Accuracy score of test: ', accuracy_score(y_test,y_test_pred))
print('Confusion Matrix of test: ', confusion_matrix(y_test,y_test_pred))
#print('Auc of test: ', roc_auc_score(y_test,y_test_prob))

So by random forest model we can find that we will get booking or not

## Checking for your booking conformed or not with whole data

In [ ]:
df1['country'].mode()

In [ ]:
df1['country'] = df1['country'].fillna('PRT')

In [ ]:
df3 = pd.get_dummies(df1,columns=['arrival_date_month','country'],drop_first=True)

In [ ]:
X = df3.drop(['is_canceled'],axis=1)
y = df3['is_canceled']

In [ ]:
params={'n_estimators':sp_randint(5,150), 'max_features':sp_randint(10,40),'max_depth':sp_randint(2,10),'min_samples_leaf':sp_randint(1,50),'min_samples_split':sp_randint(2,50),'criterion':['gini','entropy']}
rsearch=RandomizedSearchCV(rfc,param_distributions=params,n_jobs=-1,scoring='accuracy',n_iter=100,cv=3)
rsearch.fit(X,y)

In [ ]:
rsearch.best_params_

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)
rfc= RandomForestClassifier(**rsearch.best_params_)
rfc.fit(X_train,y_train)
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
y_train_pred=rfc.predict(X_train)
#y_train_prob=rfc.predict_proba(X_train)[:,1]
print('Accuracy score of train: ', accuracy_score(y_train,y_train_pred))
print('Confusion Matrix of train: ', confusion_matrix(y_train,y_train_pred))
#print('Auc of train: ', roc_auc_score(y_train,y_train_prob))
y_test_pred=rfc.predict(X_test)
#y_test_prob=rfc.predict_proba(X_test)[:,1]
print('Accuracy score of test: ', accuracy_score(y_test,y_test_pred))
print('Confusion Matrix of test: ', confusion_matrix(y_test,y_test_pred))
#print('Auc of test: ', roc_auc_score(y_test,y_test_prob))

so by using Random Forest model we can achive the best result for finding the booking is conformed or cancelled in hetel end